<a href="https://colab.research.google.com/github/loser666code/IMERSAO-AI---ALURA/blob/main/dreamCar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/loser666code/IMERSAO-AI---ALURA.git

Cloning into 'IMERSAO-AI---ALURA'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (3/3), done.


In [2]:
%pip -q install google-genai

In [3]:
import os
from google.colab import userdata
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

In [4]:
from google import genai
client = genai.Client()
MODEL_ID = "gemini-2.0-flash"
from IPython.display import HTML, Markdown

In [5]:
!pip install -q google_adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [6]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types
from datetime import date
import textwrap
from IPython.display import display, Markdown
import requests
import warnings
warnings.filterwarnings("ignore")

In [35]:
def chamarAgente(agent: Agent, message_text: str) -> str:
    session_service = InMemorySessionService()
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    content = types.Content(role="user", parts=[types.Part(text=message_text)])
    final_response = ""
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [8]:
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [45]:
def agente_analista(crt):
      analista = Agent(
      name = "Analistadecaracteristicas",
      model = "gemini-2.0-flash",
      description="Esse agente recebe todas as caracteristica do usuario e então indica todos os atributos que o carro precisa ter",
      instruction="""Você é especialista em veículos automotores especialmente em corretora de automoveis, a tarefa é analisar a lista de carracteristica de uma entidade e
      listar as necessidades e requisistos para o carro ideal para essa entidade, você está listando esses requisitos e necessidades para um especialista em automotores,
      formatada dessa maneira:
      baixo custo de manutenção : Com mecânica é simples e muitos mecânicos têm experiência com a manutenção do carro
      """
     )
      entrada_analista = f"caracteristicas : {crt}"
      lista_necessidades = chamarAgente(analista, entrada_analista)
      return lista_necessidades

In [49]:
def agente_pesquisador(lista_necessidades):
      pesquisador = Agent(
      name = "Pesquisadordecarros",
      model = "gemini-2.0-flash",
      description="Esse agente recebe todas as caracteristica do usuario e então indica todos os atributos que o carro precisa ter",
      instruction="""
      Você é especialista em veículos automotores especialmente em mecanica, analise todas as necessidades do carro e informe o carro que é possivel se compara no
      Brasil com facilidade que preencha todos esse requisitos dando prioridade a carro que preencha todos ou a maior quantidade atributos, apresentando o porque ele
      preenche esses requisitos, também os pontos positivos e negativos do carro para alguém que tem basico ou nenhum conhecimento sobre carros
      """
     )
      entrada_pesquisador = f"lista de necessidades: {lista_necessidades}"
      carro_ideal = chamarAgente(pesquisador, entrada_pesquisador)
      return carro_ideal

In [50]:
crt = input("quais as suas necessidades?")
listaNecessidades = agente_analista(crt)
display(to_markdown(listaNecessidades))
pesquisa = agente_pesquisador(listaNecessidades)
display(to_markdown(pesquisa))

quais as suas necessidades?baixo orçamento


> baixo custo de aquisição: Carros mais acessíveis no mercado de usados ou modelos de entrada novos.
> baixo custo de manutenção : Com mecânica é simples e muitos mecânicos têm experiência com a manutenção do carro
> boa economia de combustível: Para reduzir os gastos com combustível no uso diário.
> seguro acessível: Modelos com menor índice de roubo e peças de reposição mais baratas geralmente têm seguros mais em conta.
> facilidade de revenda: Escolher um modelo popular e com boa aceitação no mercado garante uma revenda mais rápida e com menor desvalorização.


> Com base nas suas necessidades, o carro que melhor se encaixa no perfil de baixo custo, manutenção acessível, economia de combustível, seguro acessível e facilidade de revenda no mercado brasileiro é o **Volkswagen Gol**.
> 
> **Por que o Gol atende a esses requisitos:**
> 
> *   **Baixo custo de aquisição:** O Gol é um dos carros mais populares no Brasil, com uma ampla oferta de modelos usados a preços acessíveis. Mesmo os modelos de entrada 0km costumam ter preços competitivos.
> *   **Baixo custo de manutenção:** A mecânica do Gol é conhecida por ser simples e robusta, o que facilita a manutenção e reduz os custos com peças e mão de obra. Além disso, a grande quantidade de Gol em circulação garante que muitos mecânicos tenham experiência com o modelo.
> *   **Boa economia de combustível:** O Gol, especialmente com motores 1.0, é um carro econômico em termos de consumo de combustível, o que ajuda a reduzir os gastos no uso diário.
> *   **Seguro acessível:** Por ser um carro popular e com menor índice de roubo em comparação com outros modelos, o Gol geralmente tem seguros mais baratos. As peças de reposição também são mais acessíveis, o que contribui para um seguro mais em conta.
> *   **Facilidade de revenda:** O Gol é um dos carros mais vendidos no Brasil, o que garante uma boa aceitação no mercado de usados e facilita a revenda. A desvalorização também costuma ser menor em comparação com outros modelos menos populares.
> 
> **Pontos positivos do Gol para iniciantes:**
> 
> *   **Fácil de dirigir:** O Gol é um carro leve e com boa dirigibilidade, o que o torna fácil de manobrar e ideal para quem está começando a dirigir.
> *   **Manutenção simples:** A mecânica do Gol é descomplicada, o que facilita a manutenção e permite que você aprenda sobre o funcionamento do carro com mais facilidade.
> *   **Peças acessíveis:** Encontrar peças de reposição para o Gol é fácil e barato, o que reduz os custos com manutenção e reparos.
> *   **Ampla oferta de modelos:** A grande variedade de modelos e versões do Gol permite que você escolha o carro que melhor se adapta às suas necessidades e orçamento.
> 
> **Pontos negativos do Gol para iniciantes:**
> 
> *   **Equipamentos básicos:** Os modelos de entrada do Gol geralmente vêm com poucos equipamentos de série, o que pode ser um problema para quem busca mais conforto e tecnologia.
> *   **Segurança:** Em termos de segurança, o Gol pode não ser o carro mais completo do mercado, especialmente nos modelos mais antigos. É importante verificar os equipamentos de segurança (airbags, freios ABS) ao escolher um modelo usado.
> *   **Design:** O design do Gol pode ser considerado um pouco datado em comparação com outros carros mais modernos, mas isso é uma questão de preferência pessoal.
> 
> **Outras opções:**
> 
> Embora o Gol seja uma excelente opção, outros carros que também podem atender às suas necessidades são:
> 
> *   **Fiat Palio/Uno:** Assim como o Gol, são carros populares, com manutenção acessível e boa economia de combustível.
> *   **Ford Ka:** Uma opção mais moderna e com design atraente, mas com manutenção um pouco mais cara que o Gol.
> *   **Volkswagen Fox:** Uma opção mais espaçosa e confortável que o Gol, mas com preço um pouco mais elevado.
> 
> Ao escolher o seu carro, é importante pesquisar bem, comparar preços e verificar o estado de conservação do veículo antes de fechar negócio.


In [51]:
!git config --global user.name "Camila Schmidt"
!git config --global user.email "programmer666loser@gamil.com"